In [2]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('D:\\NUS\\KDDM\\project_clone\\Home-sweet-home\\datasets\\transformed\\train_clean.csv')
df.head()

,rent_approval_date,town,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,region,monthly_rent,distance_to_nearest_existing_mrt,distance_to_nearest_planned_mrt,distance_to_nearest_school,distance_to_nearest_mall,coe_price_indicator,stock_price
0,0.26674,jurong east,3,new generation,0.182320,0.320755,1.344518,103.738630,yuhua east,west region,1600.0,0.271900,0.067848,0.143355,0.516103,0.17757,0.796443
1,0.26674,punggol,4,model a,0.281768,0.716981,1.398524,103.903863,matilda,north-east region,2400.0,0.265562,0.077554,0.212643,0.302355,0.17757,0.796443
2,0.26674,choa chu kang,6,premium apartment,0.596685,0.641509,1.378101,103.736002,peng siang,west region,1800.0,0.490368,0.124587,0.083780,0.401828,0.17757,0.796443
3,0.26674,sembawang,5,premium apartment,0.419890,0.660377,1.447534,103.817892,sembawang central,north region,2100.0,0.108178,0.908049,0.197100,0.081343,0.17757,0.796443
4,0.26674,bishan,5,improved,0.480663,0.396226,1.347017,103.849987,bishan east,central region,2750.0,0.167685,0.228855,0.282732,0.172704,0.17757,0.796443


In [7]:
# number of towns
df['town'].value_counts()

town
jurong west        4401
tampines           4172
sengkang           3991
bedok              3621
ang mo kio         3479
yishun             3365
bukit merah        3134
woodlands          3025
hougang            2853
punggol            2691
toa payoh          2376
clementi           2255
bukit batok        2230
choa chu kang      2114
queenstown         2013
kallang/whampoa    1848
pasir ris          1710
geylang            1708
jurong east        1704
bukit panjang      1604
sembawang          1438
bishan             1431
serangoon          1293
marine parade       609
central             606
bukit timah          56
Name: count, dtype: int64

In [49]:
df['town']

0          jurong east
1              punggol
2        choa chu kang
3            sembawang
4               bishan
             ...      
59722    bukit panjang
59723          geylang
59724           bishan
59725      jurong east
59726          geylang
Name: town, Length: 59727, dtype: object

In [9]:
df_town = pd.read_csv('D:\\NUS\\KDDM\\project_clone\\Home-sweet-home\\datasets\\town_centroids_radius.csv')
df_town.head()

,town,centroid_lat,centroid_long,town_radius_km
0,jurong west,1.341443,103.703521,3.36
1,tampines,1.345962,103.951249,2.87
2,sengkang,1.393110,103.883115,2.80
3,bedok,1.325809,103.933736,3.46
4,ang mo kio,1.382177,103.841626,2.79


In [10]:
df_town['town_radius_km'].mean()

2.7303846153846156

In [14]:
#helper function for haversine distance
import math
def haversine_distance(lat1, lon1, lat2, lon2):
    '''
    Compute the distances between 2 lat and long

    :param lat1:
    :param lon1:
    :param lat2:
    :param lon2:
    :return:
    '''
    # Radius of the Earth in kilometers
    earth_radius_km = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = earth_radius_km * c

    return distance * 1000

In [35]:
def calculate_mrts_within_town_radius(list1, list2):
    '''
    For all towns, find the number of MRTs that are within a 3km radius from town's centres

    :param list1: town lat long
    :param list2: mrt lat long
    :return:
    '''

    number_of_mrts_under_town = [0] * len(list1)
    town_counter = 0
    for point1 in list1:
        lat1, lon1 = point1
        for point2 in list2:
            lat2, lon2 = point2
            # distance = (geodesic(point1, point2).kilometers ) * 1000
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            if distance <= 2700: # if it is within a 3000m or 3km radius
                number_of_mrts_under_town[town_counter] +=1
        town_counter +=1
    return number_of_mrts_under_town

In [36]:
def calculate_schools_within_town_radius(list1, list2):
    '''
    For all towns, find the number of schools that are within a 3km radius from town's centres

    :param list1: town lat long
    :param list2: schools lat long
    :return:
    '''

    number_of_schools_under_town = [0] * len(list1)
    town_counter = 0
    for point1 in list1:
        lat1, lon1 = point1
        for point2 in list2:
            lat2, lon2 = point2
            # distance = (geodesic(point1, point2).kilometers ) * 1000
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            if distance <= 2700: # if it is within a 3000m or 3km radius
                number_of_schools_under_town[town_counter] +=1
        town_counter +=1
    return number_of_schools_under_town

In [37]:
def calculate_malls_within_town_radius(list1, list2):
    '''
    For all towns, find the number of malls that are within a 3km radius from town's centres

    :param list1: town lat long
    :param list2: malls lat long
    :return:
    '''

    number_of_malls_under_town = [0] * len(list1)
    town_counter = 0
    for point1 in list1:
        lat1, lon1 = point1
        for point2 in list2:
            lat2, lon2 = point2
            # distance = (geodesic(point1, point2).kilometers ) * 1000
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            if distance <= 2700: # if it is within a 3000m or 3km radius
                number_of_malls_under_town[town_counter] +=1
        town_counter +=1
    return number_of_malls_under_town

In [38]:
def calculate_importance_score_of_towns(number_of_mrts_under_town, number_of_mrts_planned_under_town, number_of_schools_under_town, number_of_malls_under_town):
    no_of_towns = len(number_of_mrts_under_town)
    importance_score_of_towns = []
    for i in range(no_of_towns):
        importance_score = (number_of_mrts_under_town[i] * 0.4) + (number_of_mrts_planned_under_town[i] * 0.2) + (number_of_schools_under_town[i] * 0.2) + (number_of_malls_under_town[i] * 0.2)
        importance_score_of_towns.append(importance_score)
    
    return importance_score_of_towns
    

In [39]:
mrt_input_path = "../../datasets/auxiliary-data/sg-mrt-existing-stations.csv"
mrt_planned_input_path = "../../datasets/auxiliary-data/sg-mrt-planned-stations.csv"
mall_input_path = "../../datasets/auxiliary-data/sg-shopping-malls.csv"
school_input_path = "../../datasets/auxiliary-data/sg-primary-schools.csv"

In [40]:
df_mrt = pd.read_csv(mrt_input_path)
df_mrt_planned = pd.read_csv(mrt_planned_input_path)
df_mall = pd.read_csv(mall_input_path)
df_school = pd.read_csv(school_input_path)

In [41]:
lat_long_of_towns = list(zip(df_town['centroid_lat'], df_town['centroid_long']))
lat_long_of_mrts = list(zip(df_mrt['latitude'], df_mrt['longitude']))
lat_long_of_mrts_planned = list(zip(df_mrt_planned['latitude'], df_mrt_planned['longitude']))
lat_long_of_malls = list(zip(df_mall['latitude'], df_mall['longitude']))
lat_long_of_school = list(zip(df_school['latitude'], df_school['longitude']))


In [42]:
number_of_mrts_under_town = calculate_mrts_within_town_radius(lat_long_of_towns, lat_long_of_mrts)
number_of_mrts_planned_under_town = calculate_mrts_within_town_radius(lat_long_of_towns, lat_long_of_mrts_planned)
number_of_schools_under_town = calculate_schools_within_town_radius(lat_long_of_towns, lat_long_of_school)
number_of_malls_under_town = calculate_malls_within_town_radius(lat_long_of_towns, lat_long_of_malls)

In [62]:
number_of_mrts_under_town[15]

29

In [59]:
importance_score_of_towns =  calculate_importance_score_of_towns(number_of_mrts_under_town,number_of_mrts_planned_under_town,number_of_schools_under_town,number_of_malls_under_town)
df_town['mrts'] = number_of_mrts_under_town
df_town['mrt_planned'] = number_of_mrts_planned_under_town
df_town['schools'] = number_of_schools_under_town
df_town['malls'] = number_of_malls_under_town
df_town['importance'] = importance_score_of_towns

In [60]:
df_town

,town,centroid_lat,centroid_long,town_radius_km,importance,mrts,mrt_planned,schools,malls
0,jurong west,1.341443,103.703521,3.36,7.2,3,11,13,6
1,tampines,1.345962,103.951249,2.87,9.8,10,4,18,7
2,sengkang,1.393110,103.883115,2.80,7.6,3,2,19,11
3,bedok,1.325809,103.933736,3.46,7.2,6,6,15,3
4,ang mo kio,1.382177,103.841626,2.79,4.8,5,3,8,3
5,yishun,1.417996,103.840275,2.88,3.4,2,0,10,3
6,bukit merah,1.282343,103.821103,2.69,11.8,18,2,9,12
7,woodlands,1.439668,103.788821,2.46,6.2,6,0,11,8
8,hougang,1.366029,103.890594,2.89,6.4,3,3,14,9
9,punggol,1.405524,103.909898,2.55,6.0,2,3,16,7


In [45]:
def populate_centroid_distance_matrix(lat_long_of_towns):
    centroid_distance_matrix = []
    for point1 in lat_long_of_towns:
        lat1, lon1 = point1
        distance_row = []
        for point2 in lat_long_of_towns:
            lat2, lon2 = point2
            # distance = (geodesic(point1, point2).kilometers ) * 1000
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            distance_row.append(distance)
        centroid_distance_matrix.append(distance_row)
    return  centroid_distance_matrix   

In [46]:
def calculate_centrality(importance, d, num_towns):
    # Initialize variables
    num_iterations = 100  # Number of iterations
    damping_factor = 0.85  # Damping factor for PageRank
    
    # Initialize PageRank scores
    PageRank = [1.0 / num_towns] * num_towns
    
    # Main PageRank computation loop
    for iteration in range(num_iterations):
        new_PageRank = [0] * num_towns
    
        for town_i in range(num_towns):
            new_PageRank[town_i] = (1 - damping_factor) / num_towns
    
            for town_j in range(num_towns):
                if town_i != town_j:
                    # Calculate the influence from town_j to town_i based on distance and importance
                    influence = PageRank[town_j] / (d[town_j][town_i] + 1) * importance[town_j]
                    new_PageRank[town_i] += damping_factor * influence
    
        PageRank = new_PageRank
    
    # Normalize PageRank scores
    sum_PageRank = sum(PageRank)
    PageRank = [score / sum_PageRank for score in PageRank]
    return PageRank

In [47]:
centroid_distance_matrix = populate_centroid_distance_matrix(lat_long_of_towns)
num_towns = len(df_town['town'])
centrality = calculate_centrality(importance_score_of_towns, centroid_distance_matrix,num_towns )

In [48]:
centrality

[0.03813787514013539,
 0.03830207874421084,
 0.03847978100682178,
 0.0384138752960554,
 0.03855808123843974,
 0.03833745924475194,
 0.03860170235067116,
 0.03816899271465017,
 0.03862117861086077,
 0.038326928346588386,
 0.03881941383018263,
 0.03840300282591606,
 0.038386690173183136,
 0.038235115943968126,
 0.03848575740755744,
 0.03879894459017242,
 0.03829801498820107,
 0.03872875822488231,
 0.03832333448194231,
 0.038402015260519354,
 0.03817291630758507,
 0.03864102544176496,
 0.038656509435420226,
 0.038630090063599985,
 0.03854539967493951,
 0.03852505865697996]

In [69]:
list2 = lat_long_of_mrts
number_of_mrts_under_town = 0
town_counter = 0
point1 = lat_long_of_towns[15]
print("coor: ", point1)
lat1, lon1 = point1
for point2 in list2:
    lat2, lon2 = point2
    # distance = (geodesic(point1, point2).kilometers ) * 1000
    distance = haversine_distance(lat1, lon1, lat2, lon2)
    print(distance)
    if distance <= 2700: # if it is within a 3000m or 3km radius
        #print(town_counter)
        number_of_mrts_under_town +=1
    town_counter +=1
print("result: ", number_of_mrts_under_town)

coor:  (1.313183, 103.864736)
13809.163992881491
13413.429414535456
13523.078042377747
15602.88210329881
16060.2787034491
16880.669956427664
16673.918664804394
16110.832029353212
15840.662381572733
15897.841443712054
14959.328842412193
13357.761469936495
12131.645975609437
7926.757492112411
6531.968993324507
4512.521963049628
3599.54856096392
2880.694169701379
2470.161060468806
2969.400895071047
3876.17648483273
3186.2046279811675
2617.7708505273486
2622.791993513294
3571.465145149595
4251.834365419866
4694.427760255461
11512.98571522322
9873.67286096045
10399.21049097608
9180.835067137992
7357.483427898896
5427.9165771980315
4341.233531257463
3093.0027057198927
2058.464258337895
762.9337213877484
679.5003145068728
1758.274422592497
2622.791993513294
3571.465145149595
4568.876182104942
4576.75282503178
5157.9680769307315
5943.329501672216
6842.0898307637535
11068.02503620907
8303.320004059504
9571.17958088665
11068.02503620907
13809.163992881491
15022.08812658167
16353.32401149166
1786